<a href="https://colab.research.google.com/github/Shashank251225/NETFLIX-MOVIES-AND-TV-SHOWS-CLUSTERING/blob/main/NETFLIX_MOVIES_AND_TV_SHOWS_CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - NETFLIX MOVIES AND TV SHOWS CLUSTERING



##### **Project Type**    - Unsupervised
##### **Contribution**    - Team
##### **Team Member 1 -**
##### **Team Member 2 -**
##### **Team Member 3 -**
##### **Team Member 4 -**

# **Project Summary -**

.Write the summary here within 500-600 words.

# **Problem Statement**


This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search.

In 2018, they released an interesting report which shows that the number of TV shows on Netflix has nearly tripled since 2010. The streaming service’s number of movies has decreased by more than 2,000 titles since 2010, while its number of TV shows has nearly tripled. It will be interesting to explore what all other insights can be obtained from the same dataset.

Integrating this dataset with other external datasets such as IMDB ratings, rotten tomatoes can also provide many interesting findings

#### **Define Your Business Objective?**

Exploratory Data Analysis

Understanding what type content is available in different countries

Is Netflix has increasingly focusing on TV rather than movies in recent years.

Clustering similar content by matching text-based featuresAnswer Here.

# ***Let's Begin !***

# Attribute **Information**
show_id : Unique ID for every Movie / Tv Show

type : Identifier - A Movie or TV Show

title : Title of the Movie / Tv Show

director : Director of the Movie

cast : Actors involved in the movie / show

country : Country where the movie / show was produced

date_added : Date it was added on Netflix

release_year : Actual Releaseyear of the movie / show

rating : TV Rating of the movie / show

duration : Total Duration - in minutes or number of seasons

listed_in : Genere

description: The Summary description

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE


%matplotlib inline

   


### Dataset Loading

There is a total of 7787 movies / TV shows in this data set collected from AlmaBetter

Each row contains the following information: type (Movie or TV Show), title, director, cast, country, rating (ex. PG, PG-13, R, etc.), listed_in (genre), and plot description.

In [ ]:
# Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
netflix = pd.read_csv('/content/drive/MyDrive/NETFLIX MOVIES AND TV SHOWS CLUSTERING.csv')
df = netflix.copy()


### Dataset First View

In [ ]:
# Dataset First Look
df.head()

### INFORMATION: COLUMN NAMES, MISSING VALUES AND DATA TYPES

In [ ]:
df.info()

COLUMNS

In [ ]:
df.columns

## DATASET SUMMARY

In [ ]:
df.describe(include = 'O').T

In [ ]:
df.describe().T

## MISSING VALUES

In [ ]:
df.isna().sum()

In [ ]:
#HANDLING MISSING VALUES
df['director'].fillna('unknown', inplace=True)
df['cast'].fillna('unknown', inplace=True)
df['country'].fillna('unknown', inplace=True)
df['date_added'].fillna('unknown', inplace=True)
df['rating'].fillna('unknown', inplace=True)
     

# CONVERTING 'date_added' to datetime format

In [ ]:
import datetime as dt

In [ ]:
def date(row):
  a = row['date_added']
  if 'unknown' not in a:
    month = a.split()[0]
    year = a.split()[-1]
    day = a.split()[1].split(',')[0]
    a = dt.datetime.strptime(f'{month[:3]} {day} {year}' , '%b %d %Y')
    return a
  else:
    return np.nan

df['date_added'] = df.apply(lambda x: date(x), axis=1)

## FEATURE ENGINEERING LISTED_IN 
FEATURE ENGINEERING LISTED_IN
Genres are extracted and re-defined accordingly. TV and Movie genres are separately defined. These are combined. Topics like International TV Shows are removed as it could bring in a bias by displaying content in reference to American movies.

In [ ]:
a = df.copy()

In [ ]:
a['listed_in'] = a.listed_in.apply(lambda row: row.split(', '))
a.explode('listed_in')['listed_in'].unique()
     

In [ ]:
replacing_genre = {'International TV Shows': '-',
                   'TV Dramas': 'Drama',
                   'TV Sci-Fi & Fantasy': 'SciFiFantasy',
                   'Dramas': 'Drama' ,
                   'International Movies': '-',
                   'Horror Movies': 'Horror',
                   'Action & Adventure': 'ActionAdventure',
                   'Independent Movies': '-',
                   'Sci-Fi & Fantasy': 'SciFiFantasy',
                  'TV Mysteries': 'Mystery'       ,
                  'Thrillers': 'Thriller',
                   'Crime TV Shows': 'Crime',
                   'Docuseries': 'Documentary',
                  'Documentaries': 'Documentary', 'Sports Movies': 'Sports',
                   'Comedies':'Comedy',
                   'Anime Series': 'Anime',
                  'Reality TV': 'Reality',
                   'TV Comedies': 'Comedy',
                   'Romantic Movies': 'Romance',
                  'Romantic TV Shows': 'Romance', 
                   'Science & Nature TV': 'Science',
                   'Movies': '-',
                  'British TV Shows': '-',
                   'Korean TV Shows': '-',
                   'Music & Musicals': 'Music',
                  'LGBTQ Movies': 'LGBTQ',
                   'Faith & Spirituality': 'Spirituality', 
                   "Kids' TV": 'Kids',
                  'TV Action & Adventure': 'ActionAdventure',
                   'Spanish-Language TV Shows': '-',
                  'Children & Family Movies': 'Family', 
                   'TV Shows': '-',
                   'Classic Movies': 'Classic',
                  'Cult Movies': 'Cult',
                   'TV Horror': 'Horror',
                   'Stand-Up Comedy & Talk Shows':'Comedy, TalkShow',
                  'Teen TV Shows': 'Teen', 'Stand-Up Comedy':'Comedy', 
                   'Anime Features':'Anime',
                  'TV Thrillers': 'Thriller',
                   'Classic & Cult TV':'Classic, Cult'}

In [ ]:
def genre_replace(row, replacing_genre = replacing_genre):
  new_text = []
 
  for word in (row['listed_in']):
    if word in replacing_genre:
      if '-' not in replacing_genre[word]:
       new_text.append(replacing_genre[word])
    else:
      print(word, 'not present in dictionary')

  return(', '.join(new_text))
  

df['Genres'] = a.apply(lambda row: genre_replace(row),axis=1)
df['Genres'] = df['Genres'].apply(lambda row: row.split(', '))
     

In [ ]:
(df.explode('Genres')['Genres'].unique())

#1 EXPLORATORY DATA ANALYSIS
Histogram plot of release dates of shows/movies on Netflix

In [ ]:
a = plt.figure(figsize=(14,8))
a = sns.displot(x='release_year', hue = 'type', data = df, kind = 'hist', height=10, aspect=2, bins = 50, multiple='dodge', palette='GnBu')
a = plt.xticks(ticks= np.arange(df.release_year.min(),df.release_year.max()+1,1), rotation = 90)
a = plt.title('Distribution of Movie/Shows Release Dates')
a = plt.xlabel('Release Year')

OBSERVATIONS :
Most Movies streaming on the platform were released after 2010
Most TV Shows streaming on the platform were released after 2015
The year 2017 had highest number of Movie and TV show releases on the platform

In [ ]:
df['date_added'].dt.year.max()

## Histogram plot of dates of number of shows/movies added by the Streaming giant

In [ ]:
a = plt.figure(figsize=(14,8))
a = sns.displot(x= df['date_added'], hue = 'type', data = df, kind = 'hist', height=10, aspect=2, bins = 50, multiple='dodge', palette='GnBu')
a = plt.title('Number of Movie/Shows Added by Year')
a = plt.xlabel('Added Year')

OBSERVATIONS:
Netflix began adding videos to the platform from 2008
The streaming giant started aggressively adding movies and TV shows from 2017
More movies are added as compared to TV shows

## Type of Videos on Netflix

In [ ]:
a = plt.figure(figsize=(14,8))
a = sns.countplot(x= df['type'], data = df,palette='icefire')
a = plt.title('Types of Video Entertainment on Netflix ')
a = plt.xlabel('Type')
a = plt.ylabel('Number of Shows / Movies')
     

OBSERVATIONS :
There are almost twice as many movies as TV shows on Netflix.

## Different Ratings on the Platform

In [ ]:
a = plt.figure(figsize=(14,8))
d = df.groupby('rating').agg({'show_id':'count'}).reset_index()
a = sns.barplot(x= d['rating'], y=d['show_id'], palette='bone')
a = plt.title('Ratings Distribution of Movies and Shows on Netflix')
a = plt.xlabel('Ratings')
a = plt.ylabel('Number of Videos')
     

OBSERVATIONS :
Most content on Netflix is rated for Mature Audiences and over 14 years old

## Top 10 Countries that produced content on Netflix

In [ ]:
a = df[df.country != 'unknown'].loc[:,['show_id','title','country', 'listed_in']].copy()
countrylist = (a['country'].to_list())

newlist = []
for country in countrylist:
  if ',' in country:
    a = country.split(', ')
    newlist = newlist + a
  else:
    newlist.append(country)

country_list = pd.DataFrame({'country' : newlist, 'id':np.arange(0,len(newlist),1)})
a = country_list.groupby('country').agg({'id':'count'}).sort_values('id', ascending= False).head(20).reset_index()
plt.figure(figsize=(15,10))
a = sns.barplot(y = 'country', x = 'id',data = a , palette='icefire')
a = plt.title('Movies/Show numbers produced in Countries')
a = plt.ylabel('Country')
a = plt.xlabel('Number of Videos')
top10countries = country_list.groupby('country').agg({'id':'count'}).sort_values('id', ascending= False).head(10).reset_index()['country'].unique()

OBSERVATIONS :
Top Countries in Netflix are:

1.United States
2.India 3.United Kingdom 4.Canada 5.France

## Top Genres on Netflix

In [ ]:
a = df.loc[:,['show_id','title','country', 'Genres']].copy()
a = a.explode('Genres')
genre_list = a.Genres.unique()
group = a.groupby('Genres').agg(count = ('show_id','count')).sort_values('count', ascending= False).reset_index()
plt.figure(figsize=(15,10))
_ = sns.barplot(y = 'Genres', x = 'count',data = group , palette='inferno')

In [ ]:
top15genres = group.head(15)['Genres'].to_list()

OBSERVATIONS
Top Genres in Netflix are:

Drama
2.Comedy 3.Documentary 4.Action and Adventure 5.Romance

## Frequent Directors on Netflix

In [ ]:
a=df[df.director != 'unknown'].loc[:,['show_id','title','director', 'cast']].copy()
director = (a['director'].to_list())

newlist = []
for genre in director:
  if ',' in genre:
    a = genre.split(', ')
    newlist = newlist + a
  else:
    newlist.append(genre)
country_list = pd.DataFrame({'Director' : newlist, 'id':np.arange(0,len(newlist),1)})
a = country_list.groupby('Director').agg({'id':'count'}).sort_values('id', ascending= False).reset_index().head(15)
plt.figure(figsize=(15,10))
a = sns.barplot(y = 'Director', x = 'id',data = a , palette= 'copper')
a = plt.title('Directors on Netflix')
a = plt.ylabel('Director')
a = plt.xlabel('Number of Movies/Shows Directed')
top10directors = country_list.groupby('Director').agg({'id':'count'}).sort_values('id', ascending= False).reset_index().head(10)['Director'].to_list()

OBSERVATIONS
Top Directors on Netflix are:

1.Jan Suter
2.Raul Campos 3.Marcus Raboy 4.Jay Karas 5.Cathy Garcia-Molina

## Frequent Cast on Netflix

In [ ]:
a=df[df.cast != 'unknown'].loc[:,['show_id','title','director', 'cast']].copy()
cast = (a['cast'].to_list())

newlist = []
for genre in cast:
  if ',' in genre:
    a = genre.split(', ')
    newlist = newlist + a
  else:
    newlist.append(genre)
country_list = pd.DataFrame({'Actor' : newlist, 'id':np.arange(0,len(newlist),1)})
a = country_list.groupby('Actor').agg({'id':'count'}).sort_values('id', ascending= False).head(20).reset_index()
plt.figure(figsize=(15,10))
a = sns.barplot(y = 'Actor', x = 'id',data = a , palette= 'bone')
a = plt.title('Directors on Netflix')
a = plt.ylabel('Director')
a = plt.xlabel('Number of Movies/Shows Directed')
top10actors = country_list.groupby('Actor').agg({'id':'count'}).sort_values('id', ascending= False).head(20).reset_index()['Actor'].to_list()

OBSERVATIONS
Top Actors on Netflix are:

1.Anupam Kher
2.Shah Rukh Khan 3.Naseeruddin Shah 4.Om Puri 5.Akshay Kumar

## Duration of Movies on Netflix

In [ ]:
a=df[df['type']=='Movie'].loc[:,['show_id','duration']].copy()
cast = (a['duration'].to_list())

newlist = []
for genre in cast:
  newlist.append(int(genre.split(' ')[0]))
country_list = pd.DataFrame({'Duration' : newlist, 'id':np.arange(0,len(newlist),1)})
a = sns.displot(x= 'Duration',  data = country_list, kind = 'hist', height=10, aspect=2, bins = 50)
a = plt.title('Distribution of Movie Duration')
a = plt.xlabel('Minutes')
a = plt.xticks(ticks = np.arange(0,country_list.Duration.max()+1,10))
     


OBSERVATIONS
Most movies on Netflix have a duration range from 90 to 110 minutes

## Duration of TV shows on Netflix

In [ ]:
a=df[df['type']!='Movie'].loc[:,['show_id','duration']].copy()
cast = (a['duration'].to_list())

newlist = []
for genre in cast:
  newlist.append(int(genre.split(' ')[0]))
country_list = pd.DataFrame({'Seasons' : newlist, 'id':np.arange(0,len(newlist),1)})
a = sns.displot(x= 'Seasons',  data = country_list, kind = 'hist', height=10, aspect=2)
a = plt.title('TV Show Season Distribution')
a = plt.xlabel('Seasons')
a = plt.xticks(ticks = np.arange(country_list.Seasons.min(),country_list.Seasons.max()+1,1))

OBSERVATIONS
Most TV shows on Netflix have a span of 1 season only

## 2 UNDERSTANDING CONTENT PRODUCED IN DIFFERENT COUNTRIES

In [ ]:
a = df[df['country'] != 'unknown country'].loc[:,['show_id','title','country','listed_in','Genres']].copy()

In [ ]:
#function to convert string type to list 
def operation(row,string):
  return row[string].split(', ')
a['listed_in'] = a.apply(lambda x: operation(x,'listed_in'), axis = 1)
a['country'] = a.apply(lambda x: operation(x,'country'), axis = 1)

In [ ]:
#explodes the list of categories and genres in each row to individual rows
a = a.explode('Genres').explode('country')

In [ ]:
a = a.groupby(['country','Genres']).agg({'show_id':'count'}).reset_index() #table contains number of videos belonging to each genre from different count

In [ ]:
#function to return the indeces that contain the top10 countries with the top 10 genres viewed
def selectindex(row, attribute, list):
  b = row[attribute]
  boolean = b in list
  return boolean

index = a.apply(lambda x: selectindex(x, 'country',top10countries), axis = 1)
newindex = a[index].apply(lambda x: selectindex(x, 'Genres',top15genres), axis = 1)

In [ ]:
newdata = a[index][newindex] #data that contains the number videos in the top 15 genres in the top 10 countries

In [ ]:
gyr = ['#ff00ff','#00bfff',
 '#7f0000','#483d8b','#f0e68c','#b03060','#ff7f50','#00ff00',
 '#ee82ee','#00ffff','#1e90ff','#9acd32','#00fa9a','#ffb6c1',
 '#8b008b','#b8860b','#00008b','#ffd700','#008000','#008b8b','#e0ffff',
 '#000000','#ff0000','#0000ff','#556b2f']

In [ ]:
a = plt.figure(figsize=(20,10))
sns.barplot(x = newdata['country'], y = newdata['show_id'], hue = newdata['Genres'], palette= sns.color_palette(gyr[:23]) )
plt.title('Video Content produced in the top 10 Countries')
plt.xlabel('COUNTRY')
plt.ylabel('NUMBER OF SHOWS & MOVIES')

OBSERVATIONS :
Drama is the most produced genre in a lot of Non-English speaking countries
Comedy is the most produced genre in English speaking countries like United States of America and United Kingdom and Canada
Drama and Comedy are the most produced genres in the top countries with exceptions of Japan and South Korea
Japan is the biggest producer of Anime. Anime is also the most produced in genre in Japan
Most South Korean content are from the Romance genre*
Documentaries are mainly produced in United Kingdom and United States of America

## 3 Has Netflix Been Focusing Increasingly on TV Shows as compared to movies

In [ ]:
content = df.loc[:,['title','date_added','release_year', 'duration', 'type']].dropna().copy()

In [ ]:
content['date_added'] = content['date_added'].dt.year

In [ ]:
#Assuming TV shows release new seasons every year, we'll be adding a duplicate of the show for each season.
def season(row):
  release = row['release_year']
  added = row['date_added']

  if row['type'] =='TV Show':
    seasons = int(row['duration'][0])
    b = []
    for i in range(1, seasons+1):
      if i ==1:
        b.append(release)
      else:
        b.append(b[-1]-1)
    a = b
  else:
    a = []
    a.append(row['release_year'])      
  
  return a

content['Released Years'] = content.apply(lambda x: season(x),axis=1)

In [ ]:
compare = content.explode('Released Years')

In [ ]:
tv = compare[compare['type'] == 'TV Show'].groupby(['date_added']).agg(count = ('title','count')).reset_index()
tv['type'] = 'TV'
movie = compare[compare['type'] != 'TV Show'].groupby(['date_added']).agg(count = ('title','count')).reset_index()
movie['type'] = 'Movie'

In [ ]:
data = pd.concat([tv, movie], axis = 0)

In [ ]:
plt.figure(figsize = (15,10))
sns.barplot(x = 'date_added', y='count', hue = 'type', data = data)
plt.title('MOVIE VS TV -  Trends')
plt.xlabel('Date Added')
_ = plt.ylabel('Number of TV. Seasons/Movies Signed')

OBSERVATIONS :
The above graph depicts seasons of TV shows signed vs the movies signed
This distinction gives contacts as TV shows require recurring investment for each seasons. So the TV numbers have been increased in accordance to the seasons. As they were considered as one entity earlier
We can observe that TV shows signed have been higher than movies in 2016. While the the movies signed have been higher, it is blatantly visible that the TV shows signed per year is catching up to the movies signed by the year

In [ ]:
tv = compare[compare['type'] == 'TV Show'].groupby(['Released Years']).agg(count = ('title','count')).reset_index()
tv['type'] = 'TV'
movie = compare[compare['type'] != 'TV Show'].groupby(['Released Years']).agg(count = ('title','count')).reset_index()
movie['type'] = 'Movie'
     

In [ ]:
data = pd.concat([tv, movie], axis = 0)

In [ ]:
plt.figure(figsize = (20,10))
sns.barplot(x = 'Released Years', y='count', hue = 'type', data = data)
plt.title('MOVIE VS TV -  Trends')
plt.xlabel('Released Years')
plt.xticks(rotation = 90)
_ = plt.ylabel('Number of TV. Seasons/Movies Signed')

OBSERVATIONS
The above graph depicts an estimation of seasonal TV show and Movie release date per each year. We make a general assumption that each show releases a season yearly which is often the case for most TV shows.
There are certainly exemptions. Eg. Ozark was released in years 2017, 2018, 2020 and 2022. [ P.S: Brilliant slow burn. Do check it out! ]
We can observe that TV releases outnumbering movies from 2010. It is a known fact that each show contains multiple episodes, hence we can confidently say that significant amount of video content is being enlisted under TV shows from 2010
Years 2019, 2020 and 2021 see multi-episode TV seasons outnumbering movies signed per year

## TEXT PREPROCESSING

This process involves converting the 'Description' of the show/movie into a machine interprettable form. Here we convert all text to lower case, remove punctuations and remove irrelevant words (so that only words that provide context are retained). Here similar words are unified to save memory and processing time as well. Individual words and group of words (n grams) are also collected to extract context related details.

## FEATURE ENGINEERING LISTED_IN

Genres are extracted and re-defined accordingly

In [ ]:
a = df.copy()

In [ ]:
a['listed_in'] = a.listed_in.apply(lambda row: row.split(', '))
a.explode('listed_in')['listed_in'].unique()
     

In [ ]:
replacing_genre = {'International TV Shows': '-',
                   'TV Dramas': 'Drama',
                   'TV Sci-Fi & Fantasy': 'SciFiFantasy',
                   'Dramas': 'Drama' ,
                   'International Movies': '-',
                   'Horror Movies': 'Horror',
                   'Action & Adventure': 'ActionAdventure',
                   'Independent Movies': '-',
                   'Sci-Fi & Fantasy': 'SciFiFantasy',
                  'TV Mysteries': 'Mystery'       ,
                  'Thrillers': 'Thriller',
                   'Crime TV Shows': 'Crime',
                   'Docuseries': 'Documentary',
                  'Documentaries': 'Documentary', 'Sports Movies': 'Sports',
                   'Comedies':'Comedy',
                   'Anime Series': 'Anime',
                  'Reality TV': 'Reality',
                   'TV Comedies': 'Comedy',
                   'Romantic Movies': 'Romance',
                  'Romantic TV Shows': 'Romance', 
                   'Science & Nature TV': 'Science',
                   'Movies': '-',
                  'British TV Shows': '-',
                   'Korean TV Shows': '-',
                   'Music & Musicals': 'Music',
                  'LGBTQ Movies': 'LGBTQ',
                   'Faith & Spirituality': 'Spirituality', 
                   "Kids' TV": 'Kids',
                  'TV Action & Adventure': 'ActionAdventure',
                   'Spanish-Language TV Shows': '-',
                  'Children & Family Movies': 'Family', 
                   'TV Shows': '-',
                   'Classic Movies': 'Classic',
                  'Cult Movies': 'Cult',
                   'TV Horror': 'Horror',
                   'Stand-Up Comedy & Talk Shows':'Comedy, TalkShow',
                  'Teen TV Shows': 'Teen', 'Stand-Up Comedy':'Comedy', 
                   'Anime Features':'Anime',
                  'TV Thrillers': 'Thriller',
                   'Classic & Cult TV':'Classic, Cult'}

In [ ]:
def genre_replace(row, replacing_genre = replacing_genre):
  new_text = []
 
  for word in (row['listed_in']):
    if word in replacing_genre:
      if '-' not in replacing_genre[word]:
       new_text.append(replacing_genre[word])
    else:
      print(word, 'not present in dictionary')

  return(', '.join(new_text))
  

df['Genres'] = a.apply(lambda row: genre_replace(row),axis=1)
df['Genres'] = df['Genres'].apply(lambda row: row.split(', '))

In [ ]:
(df.explode('Genres')['Genres'].unique())

In [ ]:
df['Genres']

## COMBINING Top CAST, DIRECTOR and ListedIn into single text attributes

We are extracting non-plot related text details about the movie/show like lead actors, director, country and categories it is being listed in on the platform

In [ ]:
def combine(row):
  text = ' '.join(row['Genres']).lower()
  if row['cast'] != 'unknown':
    cast_list = [actor.replace(' ', '') for actor in row['cast'].split(', ')]
    if len(cast_list) < 5:
      text = text + ' ' + ' '.join(cast_list[:]).lower()
    else :
      text = text + ' ' + ' '.join(cast_list[:5]).lower()
  if row['director'] !='unknown':
    director_list = ' '.join([director.replace(' ', '') for director in row['director'].split(', ')])
    text = text + ' ' + director_list.lower()

  return text
df['Movie Deets'] = df.apply(lambda row: combine(row), axis = 1)

In [ ]:
df.head()['Movie Deets'].to_list()

In [ ]:
#importing necessary libraries for text processing
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('punkt')

In [ ]:
 nltk.download('stopwords')

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

## WORD TOKENIZATION

Tokenization breaks the raw text into words, sentences called tokens. These tokens help in understanding the context or developing the model for the NLP. The tokenization helps in interpreting the meaning of the text by analyzing the sequence of the words.

In [ ]:
import string

In [ ]:
def tokenizer(txt):  
  tokens = word_tokenize(txt) 
  
  return tokens
df['Processed Description'] = df['description'].apply(tokenizer)
df['Processed Movie Deets'] = df['Movie Deets'].apply(tokenizer)

## PUNCTUATION REMOVAL

In [ ]:
regular_punct = list(string.punctuation)
def remove_punctuation(text,punct_list):
  tokens = text
  new_tok = []
  for i in tokens:
    word = i
    for punctuation in list(string.punctuation):
      word = word.replace(punctuation, ' ')
    if word.replace(' ','').isalnum():
      new_tok.append(word)

  return ' '.join(new_tok).split(' ')

df['Processed Description'] = df['Processed Description'].apply(lambda x : remove_punctuation(x, regular_punct))
df['Processed Movie Deets'] = df['Processed Movie Deets'].apply(lambda x : remove_punctuation(x, regular_punct))


## STOP WORD REMOVAL

In [ ]:
stop_words = set(stopwords.words('english')) 

def remove_stop(tokens, stopwords):
  return [t.lower() for t in tokens if t.lower() not in stopwords]


df['Processed Description'] = df['Processed Description'].apply(lambda x : remove_stop(x, stop_words))

a = df.copy() #will be used for wordcloud later on
     

## STEMMING
Stemming is the process of producing morphological variants of a root/base word. Stemming programs are commonly referred to as stemming algorithms or stemmers.

In [ ]:
ps = PorterStemmer()

def stem_text(text):  
  stemmed = []
  for word in text:
    stemmed.append(ps.stem(word))
  return stemmed


df['Processed Description'] = df['Processed Description'] .apply(lambda x : stem_text(x))

In [ ]:
def joiner(text_list):
  new = []
  for text in text_list:
    if text == ' ' or text == '' :
      pass
    else:
      new.append(text)
      
  return ' '.join(new)

df['Processed Description'] = df['Processed Description'] .apply(lambda x : joiner(x))
df['Processed Movie Deets'] = df['Processed Movie Deets'].apply(lambda x : joiner(x))
     

In [ ]:
df.loc[:,['description','Processed Description','Movie Deets',	'Processed Movie Deets']].head(1).T

## Vectorizing Texts

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
count_vectorizer = CountVectorizer(min_df = 10,max_df=0.5)
count_description_vector = count_vectorizer.fit_transform(df['Processed Description'])

count_movie_vector = count_vectorizer.fit_transform(df['Processed Movie Deets'])

In [ ]:
count_description_vector.shape

In [ ]:
count_movie_vector.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(pd.concat([pd.DataFrame(count_description_vector.todense()), pd.DataFrame(count_movie_vector.todense())],axis =1))

In [ ]:
exp_var_pca = pca.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.figure(figsize = (20,10))
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.xticks(ticks = np.arange(0,2300,100))
plt.axhline(y=0.75)
plt.show()

Vectorising the preprocessed attributes Movie Deets and Description has sum total of 2126 + 192 = 2318 dimensions. These dimensions will have to be reduced using PCA, which would result in loss of information. Alternatively, the two attributes can be used to model the content into topics using Latent Dirichlet Allocation. This would makes sure that all the topical information about video content are captured without putting any available information to waste.

## BAG OF WORDS WORDCLOUD GENERATION of CONTENT DESCRIPTION BASED ON GENRE

In [ ]:
from wordcloud import WordCloud

In [ ]:
genres = df.explode('Genres').Genres.unique()

In [ ]:
copy =  a.explode('Genres').copy()
a = copy.copy()
a['Processed Description'] = a['Processed Description'] .apply(lambda x : joiner(x))

In [ ]:
plt.figure(figsize=(35, 30))
plt.suptitle("Word Cloud for Different Genres", fontsize=18, y=0.5)

for genre in top15genres:
  if genre != '':
    
    count_description_vectorizer = CountVectorizer(ngram_range = (1,1), min_df = 5, max_df = 0.7)
    # fit the count vectorizer using the text data
    count_description_vectorizer.fit(a[a.Genres == genre]['Processed Description'])
    dictionary = count_description_vectorizer.vocabulary_.items()

    vocab = []
    count_of_vocab = []
    # Iterate through each vocab and count append the value to designated lists
    for key, value in dictionary:
        vocab.append(key)
        count_of_vocab.append(value)

    count_description_df = pd.DataFrame({'Token':vocab, 'Count':np.array(count_of_vocab)}).sort_values('Count', ascending= False)
    comment_words = ' '.join(count_description_df['Token'].to_list())
    
    wordcloud = WordCloud(width = 1200, height = 800,
                    background_color ='black',
                    min_font_size = 10, random_state = 33).generate(comment_words)
                    # plot the WordCloud image 
    plt.figure(figsize = (12, 8), facecolor = None)                     
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.title(genre.upper())
    
    plt.show()

## BIGRAM BAG OF WORDS WORDCLOUD GENERATION of CONTENT DESCRIPTION BASED ON GENRE

In [ ]:
a = copy.copy()
a['Processed Description'] = a['Processed Description'] .apply(lambda x : joiner(x))

In [ ]:
plt.figure(figsize=(35, 30))
plt.suptitle("Word Cloud for Different Genres", fontsize=18, y=0.5)

for genre in top15genres:
  if genre != '':
    
    count_description_vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 1, max_df = 0.7)
    # fit the count vectorizer using the text data
    count_description_vectorizer.fit(a[a.Genres == genre]['Processed Description'])
    dictionary = count_description_vectorizer.vocabulary_.items()

    vocab = []
    count_of_vocab = []
    # Iterate through each vocab and count append the value to designated lists
    for key, value in dictionary:
        vocab.append(key)
        count_of_vocab.append(value)


    count_description_df = pd.DataFrame({'Token':vocab, 'Count':np.array(count_of_vocab)}).sort_values('Count', ascending = False)
    comment_words = ' '.join(count_description_df['Token'].apply(lambda row: ''.join(row.split(' '))).to_list())
        
    wordcloud = WordCloud(width = 1200, height = 800,
                    background_color ='black',
                    min_font_size = 10, random_state = 33).generate(comment_words)
                    # plot the WordCloud image 
    plt.figure(figsize = (12, 8), facecolor = None)                     
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.title(genre.upper())
    
    plt.show()

## Preparing dataset for clustering

In [ ]:
clustering_df = df[['type', 'duration','rating','release_year']]

Encoding TV and Movie ratings into one based on their age reccomendations

Guidelines:

Amazon: Maturity Rating Guidelines

Netflix: Maturity Rating Guidelines

In [ ]:
def topick(row):
  older_kids = ['TV-PG', 'PG', 'TV-Y7', 'TV-Y7-FV']
  kids = ['TV-G', 'G', 'TV-Y']
  teens = ['PG-13','TV-14']
  adult = ['R', 'NC-17', 'TV-MA', 'UR', 'NR']

  rating = 'Others'
  for column in older_kids + kids + teens + adult:
    if row[column] == 1:
      if column in older_kids:
        rating = 'Older Kids'
      elif column in kids :
        rating = 'Kids'
      elif column in teens :
        rating = 'Teens'
      elif column in adult:
        rating = 'Adult'
      
  return rating

ratings = pd.get_dummies(clustering_df['rating']).drop('unknown',axis=1).apply(lambda x: topick(x),axis=1)
     

## ENCODING TYPE, RATING

In [ ]:
clustering_df = pd.concat([clustering_df.drop(['type','rating'],axis =1),pd.get_dummies(ratings).drop('Others', axis=1), pd.get_dummies(clustering_df['type'])], axis =1)
     

In [ ]:
clustering_df

## PROCESSING MOVIE DURATION

Extracting season numbers and movie duration as separate columns

In [ ]:
def duration(row,column):
  text = row['duration'].split()
  if column == 'show':
    if row['Movie'] == 0:
      return int(text[0])
    else:
      return 0
  if column == 'movie':
    if row['Movie'] ==1:
      return int(text[0])
    else:
      return 0
clustering_df['show_duration'] = clustering_df.apply(lambda x: duration(x,'show'),axis =1)

clustering_df['movie_duration'] = clustering_df.apply(lambda x: duration(x,'movie'), axis=1)

In [ ]:
clustering_df.drop('duration', axis = 1, inplace=True)

## Binning Release Years based on decades

In [ ]:
(clustering_df['release_year']).unique()

In [ ]:
def bin_release_year(row):
  r_year = row['release_year']
  for yr in np.arange(2030,1910,-10):
    if r_year >= yr:
      return (yr)

clustering_df['release_year_bins'] =(clustering_df.apply(lambda x: bin_release_year(x), axis=1))

In [ ]:
clustering_df

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer


import matplotlib.cm as cm
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


from sklearn.cluster import KMeans , AgglomerativeClustering, DBSCAN
from sklearn.cluster import AffinityPropagation
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder


## TOPIC MODELING using GENSIM

In [ ]:
topic_input = df['Processed Description'] + ' ' + df['Processed Movie Deets']

In [ ]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
from gensim.corpora import Dictionary

gensim_paragraphs = (topic_input).apply(lambda row: row.split(' '))
dict_gensim_para = Dictionary(gensim_paragraphs) # dictionary of corpus
dict_gensim_para.filter_extremes(no_below=5, no_above=0.6)

bow_gensim_para = [dict_gensim_para.doc2bow(paragraph) for paragraph in gensim_paragraphs] #creating corpus

## TFIDF VECTOR

In [ ]:
from gensim.models import TfidfModel
tfidf_gensim_para = TfidfModel(bow_gensim_para)
vectors_gensim_para = tfidf_gensim_para[bow_gensim_para]

In [ ]:
lda_para_model_n = []
for n in (range(5, 16)):
  lda_model = LdaMulticore(corpus=bow_gensim_para, id2word=dict_gensim_para,
                            chunksize=1000, eta='auto', iterations=400,
                            num_topics=n, passes=20, eval_every=None,
                            random_state=42)
  lda_coherence = CoherenceModel(model=lda_model, texts=gensim_paragraphs,
                            dictionary=dict_gensim_para, coherence='c_npmi') 
  print(n,'|', lda_coherence.get_coherence())
  lda_para_model_n.append((n, lda_model, lda_coherence.get_coherence()))

In [ ]:
lda_df = pd.DataFrame(lda_para_model_n, columns=["n", "model", "coherence"]).set_index("n")[["coherence"]]
plt.figure(figsize=(16,9))
_ = sns.lineplot(x='n', y='coherence', data=lda_df )
_ = plt.xticks(ticks = np.arange(4,25,1))

In [ ]:
lda_df = lda_df.reset_index()
topic_nums = lda_df[lda_df['coherence']==lda_df['coherence'].max()]['n'].to_list()[0]
topic_nums

In [ ]:
lda_gensim_para = LdaMulticore(corpus=bow_gensim_para, id2word=dict_gensim_para,
                            chunksize=2000, eta='auto', iterations=400,
                            num_topics= topic_nums , passes=20, eval_every=None,
                            random_state=42)

In [ ]:
#function to display gensim topics
def display_gensim_topics(model):
  gen_list = lda_gensim_para.print_topics()
  for i in range(0,len(gen_list)):
    print('Topic: ',i+1,'\n')
    topics = gen_list[i][1]
    for j in topics.split('+'):
      print(j.split('"')[1])

    print("------\n")

display_gensim_topics(lda_gensim_para)

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(lda_gensim_para, bow_gensim_para, dict_gensim_para)

p

In [ ]:
lda = {}
for i in range(0,lda_gensim_para.num_topics):
  lda[f'Topic {i}'] = np.zeros(df.shape[0])

In [ ]:
for i, row in enumerate(lda_gensim_para[bow_gensim_para]):
  for topic in row:
    lda[f'Topic {topic[0]}'][i] = topic[1] 

In [ ]:
topic_model_df = pd.DataFrame(lda)

In [ ]:
topic_model_df

In [ ]:
def topic_input(row):
  highest = 0
  top = 'None'
  for topic in topic_model_df.columns:
    if row[topic] > highest:
      highest = row[topic]
      top = f'Topic {int(topic[-1])+1}'
  return top

topic = topic_model_df.apply(lambda x:topic_input(x),axis=1)

# CREATING WORD CLOUDS FOR TOPICS

In [ ]:
a = copy.copy()
a['Processed Description'] = a['Processed Description'] .apply(lambda x : joiner(x))
a = pd.concat([a,pd.DataFrame(topic, columns = ['Topic'])],axis=1)

In [ ]:
plt.figure(figsize=(35, 30))
plt.suptitle("Word Cloud for Different Genres", fontsize=18, y=0.5)

for top in topic.unique():

  count_description_vectorizer = CountVectorizer(ngram_range = (1,2), min_df = 5, max_df = 0.7)
  # fit the count vectorizer using the text data
  count_description_vectorizer.fit(a[a['Topic'] == top]['Processed Description'])
  dictionary = count_description_vectorizer.vocabulary_.items()

  vocab = []
  count_of_vocab = []
  # Iterate through each vocab and count append the value to designated lists
  for key, value in dictionary:
      vocab.append(key)
      count_of_vocab.append(value)


  count_description_df = pd.DataFrame({'Token':vocab, 'Count':np.array(count_of_vocab)}).sort_values('Count', ascending = False)
  comment_words = ' '.join(count_description_df['Token'].apply(lambda row: ''.join(row.split(' '))).to_list())
      
  wordcloud = WordCloud(width = 1200, height = 800,
                  background_color ='black',
                  min_font_size = 10).generate(comment_words)
                  # plot the WordCloud image 
  plt.figure(figsize = (12, 8), facecolor = None)                     
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  plt.title(top.upper())
  
  plt.show()

## Concatenating clustering_df with topic models

In [ ]:
clustering_df

## SCALING

In [ ]:
#vectorizer_clustering = pd.concat([clustering_df.drop(['release_year','Movie','TV Show'], axis=1), pd.get_dummies(clustering_df['release_year'])], a

In [ ]:
vectorizer_clustering = pd.concat([clustering_df.drop(['release_year'], axis=1),], axis =1)

In [ ]:
vectorizer_clustering

In [ ]:
scaler = MinMaxScaler()
X_rescale = scaler.fit_transform(vectorizer_clustering)
#dataset contains degree of belonging of document to each topic concatenated to scaled Dataframe 
X = pd.concat([pd.DataFrame(X_rescale,columns = vectorizer_clustering.columns ),(topic_model_df)],axis=1)

In [ ]:
X

In [ ]:
X = X.to_numpy()

In [ ]:
X.shape

## Performing Hopkins test for uniform distribution to check if clustering is possible on the dataset

In [ ]:
pip install pyclustertend

In [ ]:
pip install numba==0.53

In [ ]:
from pyclustertend import hopkins
hopkins(X,150)

In [ ]:
X.shape

## CREATING DICTIONARY FOR REPORT

In [ ]:
clustering_report = {
    'Algorithm':[],
    'Parameters':[],
    'Clusters':[],
    'Silhouette Coefficient':[],
    'Davies-Bouldin Index': [],
    'Calinski-Harbaz Score': []
}

In [ ]:
def update_report(algo_name, details,clusters, sil_score, db_index, ch_score):
  data = [algo_name, details, clusters,sil_score, db_index, ch_score]
  for i,detail in enumerate(clustering_report):
    clustering_report[detail].append(data[i])

## DBSCAN

DBSCAN stands for Density-Based Spatial Clustering of Applications with Noise.It groups ‘densely grouped’ data points into a single cluster. It can identify clusters in large spatial datasets by looking at the local density of the data points. The most exciting feature of DBSCAN clustering is that it is robust to outliers. It also does not require the number of clusters to be told beforehand, unlike K-Means, where we have to specify the number of centroids.

DBSCAN requires only two parameters: epsilon and minPoints. Epsilon is the radius of the circle to be created around each data point to check the density and minPoints is the minimum number of data points required inside that circle for that data point to be classified as a Core point.

FITTING AND PREDICTING DATA WITH DBSCAN with default values

In [ ]:
dbscan = DBSCAN().fit(X)
preds = dbscan.labels_

CALCULATING SILHOUETTE SCORES

In [ ]:
silhouette_score(X,preds)

In [ ]:
davies_bouldin_score(X,preds)

In [ ]:
calinski_harabasz_score(X,preds)

NUMBER OF CLUSTERS

In [ ]:
pd.Series(preds).nunique()

OBSERVATIONS
DBSCAN clustered the data into 9 clusters with a silhouette score 0.4968

VISUALIZATION
For visualization, we're taking three significant components using PCA and plotting them 2 the front,side and top view of the 3d space. This way we can see how the clusters are arranged spatially

In [ ]:
predictions = pd.DataFrame(preds, columns = ['Labels'])

In [ ]:
# Dimensionality reduction using PCA
pca = PCA(n_components = 3)
     

In [ ]:
reduced = pd.DataFrame(pca.fit_transform(X), columns = ['Dimension 1', 'Dimension 2', 'Dimension 3'])
dbscan_cluster = pd.concat([reduced, predictions], axis=1)

In [ ]:
#setting colors for all visualisations
hundred_colors = " #000000 dimgray #696969 silver #c0c0c0 darkslategray #2f4f4f darkolivegreen #556b2f olivedrab #6b8e23 sienna #a0522d seagreen #2e8b57 forestgreen #228b22 maroon2 #7f0000 midnightblue #191970 darkgreen #006400 olive #808000 darkslateblue #483d8b firebrick #b22222 lightslategray #778899 mediumseagreen #3cb371 rosybrown #bc8f8f rebeccapurple #663399 darkgoldenrod #b8860b darkkhaki #bdb76b darkcyan #008b8b steelblue #4682b4 chocolate #d2691e yellowgreen #9acd32 darkblue #00008b indigo #4b0082 limegreen #32cd32 darkseagreen #8fbc8f darkmagenta #8b008b maroon3 #b03060 mediumturquoise #48d1cc mediumaquamarine #66cdaa darkorchid #9932cc orangered #ff4500 orange #ffa500 gold #ffd700 yellow #ffff00 mediumvioletred #c71585 mediumblue #0000cd burlywood #deb887 chartreuse #7fff00 lime #00ff00 mediumorchid #ba55d3 springgreen #00ff7f darksalmon #e9967a crimson #dc143c aqua #00ffff deepskyblue #00bfff sandybrown #f4a460 mediumpurple #9370db blue #0000ff purple3 #a020f0 lightcoral #f08080 greenyellow #adff2f tomato #ff6347 thistle #d8bfd8 fuchsia #ff00ff palevioletred #db7093 khaki #f0e68c laserlemon #ffff54 cornflower #6495ed plum #dda0dd lightgreen #90ee90 lightblue #add8e6 deeppink #ff1493 mediumslateblue #7b68ee violet #ee82ee lightskyblue #87cefa aquamarine #7fffd4 lightgoldenrod #fafad2 hotpink #ff69b4 mistyrose #ffe4e1 lightcyan #e0ffff pink #ffc0cb"
colors = [color for color in hundred_colors.split(' ') if '#' in color]

#looping to shuffle colors
np.random.seed(151203)
shuffled = []
while len(colors)!=0:
  a = np.random.choice(colors)
  shuffled.append(a)
  colors.remove(a)

In [ ]:
gyr = shuffled[:pd.Series(preds).nunique()]

In [ ]:
sns.color_palette(gyr)

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("3 Dimensional PCA Cluster Visualizations", fontsize=18, y=0.95)

plt.subplot(2,2,1)
plt.title('TOP VIEW')
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 2', data = dbscan_cluster,palette = sns.color_palette(gyr), hue = 'Labels', legend=False)

plt.subplot(2,2,2)
plt.title('SIDE VIEW')
sns.scatterplot(x = 'Dimension 3', y = 'Dimension 2',  data = dbscan_cluster,palette = sns.color_palette(gyr), hue = 'Labels', legend=False)

plt.subplot(2,2,3) 
plt.title('FRONT VIEW')
palette = {0: "C0", 1: "C1", 2: "C2", 3: "C3", 4: "C4", 5: "C5",
           6: "C6", 7: "C7", 8: "C8", 9: "C9", 10: "C10", 11: "C11",
           12: "C12", 13: "C13", 14: "C14", 15: "C15", 16: "C16"}
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 3', data = dbscan_cluster,palette = sns.color_palette(gyr), hue = 'Labels',legend=False)

## K-MEANS
It is an iterative algorithm that divides the unlabeled dataset into k different clusters in such a way that each dataset belongs only one group that has similar propert

Implementing K-means clustering and calculating the Silhouette scores for clusters of 2 to 25 for content based on Release Year, Ratings, Type and Duration

A silhouette score of 1 means that the clusters are very dense and nicely separated. The score of 0 means that clusters are overlapping. The score of less than 0 means that data belonging to clusters may be wrong/incorrect. The silhouette plots can be used to select the most optimal value of the K

In [ ]:
silhouette_score_ = [ ]
sum_of_sq_dist = {}
range_n_clusters = [i for i in range(2,21)]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, n_init=10,random_state=10)
    preds = clusterer.fit_predict(X)
    centers = clusterer.cluster_centers_
    sum_of_sq_dist[n_clusters] = clusterer.inertia_

    score = silhouette_score(X, preds)
    silhouette_score_.append([int(n_clusters) , round(score , 3)])

In [ ]:
Kmeans = pd.DataFrame(silhouette_score_ , columns = ["No of Clusters" , "Silhouette Score"])

## Clusters vs Silhouette score

In [ ]:
plt.figure(figsize=(25,10))
_ = sns.lineplot(x = Kmeans['No of Clusters'], y = Kmeans['Silhouette Score'])
_ = plt.xticks(ticks = np.arange(0,22,1))

In [ ]:
Kmeans.sort_values(['Silhouette Score','No of Clusters'],ascending=False).head()


OBSERVATIONS
The highest silhouette score is 0.460 produced with 8 Clusters using K Means Algorithm

## Using Elbow Method to determine the Optimal number of Clusters

In [ ]:
#Plot the graph for the sum of square distance values and Number of Clusters
plt.figure(figsize=(25,8))
sns.pointplot(x = list(sum_of_sq_dist.keys()), y = list(sum_of_sq_dist.values()))
plt.xlabel('Number of Clusters(k)')
plt.ylabel('Sum of Square Distances')
plt.title('Elbow Method For Optimal k')
plt.xticks(rotation = 90)
plt.show()

OBSERVATIONS
From the above graph has plots of the sum squared inertia for K clusters trained on K means algorithm
In this graph, we decide the optimal number of clusters by locating the elbow of the graph which is at 8 clusters

## Displaying Silhouette plot for K means Clustering

In [ ]:
range_n_clusters = [i for i in range(2,20)]
silhouette_score_ = []
for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) /n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")
    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

OBSERVATIONS
From Silhouette Analysis and Elbow method, the optimal cluster is 8. This gives a clustering score of 0.389

## VISUALIZATION for K = 8 clusters
For visualization, we're taking three significant components using PCA and plotting them 2 the front,side and top view of the 3d space. This way we can see how the clusters are arranged spatially

In [ ]:
clusterer = KMeans(n_clusters= 8, n_init=10,random_state=10)
preds = clusterer.fit_predict(X)

In [ ]:
update_report('KMeans','Default' ,pd.Series(preds).nunique(), silhouette_score(X,preds), davies_bouldin_score(X,preds), calinski_harabasz_score(X,preds))
     

In [ ]:
predictions = pd.DataFrame(preds, columns = ['Clusters'])

In [ ]:
# Dimensionality reduction using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)


In [ ]:
reduced = pd.DataFrame(pca.fit_transform(X), columns = ['Dimension 1', 'Dimension 2', 'Dimension 3'])
kmeans_cluster = pd.concat([reduced, predictions], axis=1)
     

In [ ]:
#setting colors for all visualisations
hundred_colors = " #000000 dimgray #696969 silver #c0c0c0 darkslategray #2f4f4f darkolivegreen #556b2f olivedrab #6b8e23 sienna #a0522d seagreen #2e8b57 forestgreen #228b22 maroon2 #7f0000 midnightblue #191970 darkgreen #006400 olive #808000 darkslateblue #483d8b firebrick #b22222 lightslategray #778899 mediumseagreen #3cb371 rosybrown #bc8f8f rebeccapurple #663399 darkgoldenrod #b8860b darkkhaki #bdb76b darkcyan #008b8b steelblue #4682b4 chocolate #d2691e yellowgreen #9acd32 darkblue #00008b indigo #4b0082 limegreen #32cd32 darkseagreen #8fbc8f darkmagenta #8b008b maroon3 #b03060 mediumturquoise #48d1cc mediumaquamarine #66cdaa darkorchid #9932cc orangered #ff4500 orange #ffa500 gold #ffd700 yellow #ffff00 mediumvioletred #c71585 mediumblue #0000cd burlywood #deb887 chartreuse #7fff00 lime #00ff00 mediumorchid #ba55d3 springgreen #00ff7f darksalmon #e9967a crimson #dc143c aqua #00ffff deepskyblue #00bfff sandybrown #f4a460 mediumpurple #9370db blue #0000ff purple3 #a020f0 lightcoral #f08080 greenyellow #adff2f tomato #ff6347 thistle #d8bfd8 fuchsia #ff00ff palevioletred #db7093 khaki #f0e68c laserlemon #ffff54 cornflower #6495ed plum #dda0dd lightgreen #90ee90 lightblue #add8e6 deeppink #ff1493 mediumslateblue #7b68ee violet #ee82ee lightskyblue #87cefa aquamarine #7fffd4 lightgoldenrod #fafad2 hotpink #ff69b4 mistyrose #ffe4e1 lightcyan #e0ffff pink #ffc0cb"
colors = [color for color in hundred_colors.split(' ') if '#' in color]

#looping to shuffle colors
np.random.seed(151203)
shuffled = []
while len(colors)!=0:
  a = np.random.choice(colors)
  shuffled.append(a)
  colors.remove(a)

In [ ]:
gyr = shuffled[:pd.Series(preds).nunique()]

In [ ]:
sns.color_palette(gyr)

In [ ]:
from matplotlib import transforms
plt.figure(figsize=(20,15))
plt.suptitle("3 Dimensional PCA Cluster Visualizations", fontsize=18, y=0.95)

plt.subplot(2,2,1)
plt.title('TOP VIEW')
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 2', data = kmeans_cluster, hue = 'Clusters',palette = sns.color_palette(gyr), legend=False
                )

plt.subplot(2,2,2)
plt.title('SIDE VIEW')
palette = {0: "C0", 1: "C1", 2: "C2", 3: "C3", 4: "C4", 5: "C5",
           6: "C6", 7: "C7", 8: "C8", 9: "C9", 10: "C10", 11: "C11",
           12: "C12", 13: "C13", 14: "C14", 15: "C15", 16: "C16"}
sns.scatterplot(x = 'Dimension 3', y = 'Dimension 2', data = kmeans_cluster, hue = 'Clusters',palette = sns.color_palette(gyr), legend=False)

plt.subplot(2,2,3) 
plt.title('FRONT VIEW')
palette = {0: "C0", 1: "C1", 2: "C2", 3: "C3", 4: "C4", 5: "C5",
           6: "C6", 7: "C7", 8: "C8", 9: "C9", 10: "C10", 11: "C11",
           12: "C12", 13: "C13", 14: "C14", 15: "C15", 16: "C16"}
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 3', data = kmeans_cluster, hue = 'Clusters',palette = sns.color_palette(gyr))
plt.legend(bbox_to_anchor=(1.5,1))

## HIERARCHICAL CLUSTERING
Hierarchical clustering, also known as hierarchical cluster analysis, is an algorithm that groups similar objects into groups called clusters. The endpoint is a set of clusters, where each cluster is distinct from each other cluster, and the objects within each cluster are broadly similar to each other.

In [ ]:
# Let's import sch
import scipy.cluster.hierarchy as sch
hier_ward_euc = sch.linkage(X, method = 'ward', metric = 'euclidean')

## DENDROGRAM

In [ ]:
plt.figure(figsize=(30,15))
dendrogram = sch.dendrogram(hier_ward_euc)
plt.title('Dendrogram')
plt.xlabel('Content')
plt.ylabel('Euclidean Distances')
plt.show() # find largest vertical distance we can make without crossing any other horizontal line

FROM DENDOGRAM, A DISTANCE OF 20-30 appears to have well defined trees.

CHOOSING THE APPROPRIATE DISTANCE TO CLUSTER

## Agglomerative Clustering

In [ ]:
clusters = []
scores = []
distances = []
for i in np.arange(5,75,5):
  hc = AgglomerativeClustering(n_clusters = None, distance_threshold = i, affinity = 'euclidean', linkage = 'ward')
  y_hc = hc.fit_predict(X)
  clusters.append(hc.labels_.max()+1)
  if (hc.labels_.max()+1 >=2) :
    scores.append(silhouette_score(X, y_hc))
  else:
    scores.append(1)
  distances.append(i)

hierarchical = pd.DataFrame({'Clusters': clusters, 
                             'Silhouette Scores': scores,
                             'Distance': distances})
     

DISTANCE vs SILHOUETTE SCORES GRAPH FOR HIERARCHICAL CLUSTERING

In [ ]:
plt.figure(figsize =(15,8))
sns.lineplot(x = 'Distance', y = 'Silhouette Scores' , data = hierarchical)
a = plt.title('Hierarchical Clustering: Distance vs Silhouette Scores')
_ = plt.xticks(np.arange(5,75,1),rotation = 90)

Distance-Silhouette Score-Clusters Table for Hierarchical Clustering

In [ ]:
hierarchical.sort_values('Silhouette Scores', ascending = False).head()

OBSERVATIONS :
Highest Silhouette Score of 0.498 achieved at distance = 20 with 8 clusters

In [ ]:
plt.figure(figsize =(30,15))
plt.title("Dendrograms")  
dend = sch.dendrogram(sch.linkage(X, method='ward'))
plt.axhline(y= 20, color='black', linestyle='--')

## VISUALIZATION for Distance = 20
For visualization, we're taking three significant components using PCA and plotting them 2 the front,side and top view of the 3d space. This way we can see how the clusters are arranged spatially

In [ ]:
hc = AgglomerativeClustering(n_clusters = None, distance_threshold = 20, affinity = 'euclidean', linkage = 'ward')
preds = hc.fit_predict(X)

In [ ]:
update_report('Hierarchical Agglomerative','Distance = 20' ,pd.Series(preds).nunique(), silhouette_score(X,preds), davies_bouldin_score(X,preds), calinski_harabasz_score(X,preds))

In [ ]:
predictions = pd.DataFrame(preds, columns = ['Labels'])

In [ ]:
# Dimensionality reduction using PCA
pca = PCA(n_components = 3)

In [ ]:
reduced = pd.DataFrame(pca.fit_transform(X), columns = ['Dimension 1', 'Dimension 2', 'Dimension 3'])
hier_cluster = pd.concat([reduced, predictions], axis=1)

In [ ]:
colors = shuffled[:pd.Series(hc.labels_).nunique()]

sns.color_palette(colors)

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("3 Dimensional PCA Cluster Visualizations", fontsize=18, y=0.95)

plt.subplot(2,2,1)
plt.title('TOP VIEW')
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 2', data = hier_cluster, hue = 'Labels',palette = sns.color_palette(colors), legend=False)

plt.subplot(2,2,2)
plt.title('SIDE VIEW')
palette = {0: "C0", 1: "C1", 2: "C2", 3: "C3", 4: "C4", 5: "C5",
           6: "C6", 7: "C7", 8: "C8", 9: "C9", 10: "C10", 11: "C11",
           12: "C12", 13: "C13", 14: "C14", 15: "C15", 16: "C16"}
sns.scatterplot(x = 'Dimension 3', y = 'Dimension 2',  data = hier_cluster, hue = 'Labels',palette = sns.color_palette(colors), legend=False)

plt.subplot(2,2,3) 
plt.title('FRONT VIEW')
palette = {0: "C0", 1: "C1", 2: "C2", 3: "C3", 4: "C4", 5: "C5",
           6: "C6", 7: "C7", 8: "C8", 9: "C9", 10: "C10", 11: "C11",
           12: "C12", 13: "C13", 14: "C14", 15: "C15", 16: "C16"}
sns.scatterplot(x = 'Dimension 1', y = 'Dimension 3', data = hier_cluster, hue = 'Labels',palette = sns.color_palette(colors), legend=False)

## FINAL REPORT
After optimizing and selecting the best number of clusters for the three models, the final report is displayed down below

In [ ]:
pd.DataFrame(clustering_report)

# **CONCLUSIONS**
#1. Exploratory Data Analysis Conclusions
Netflix began adding videos to the platform from 2008
The streaming giant started aggressively adding movies and TV shows from 2017
More movies are added as compared to TV shows
There are almost twice as many movies as TV shows on Netflix.
Most content on Netflix is rated for Mature Audiences and over 14 years old

**Top Countries in Netflix are**:

United States
India
United Kingdom
Canada
France

**Top Genres in Netflix are:**

1Drama
2.Comedy 3.Documentary 4.Action and Adventure 5.Romance

**Top Directors on Netflix are**:

1Jan Suter
2.Raul Campos 3.Marcus Raboy 4.Jay Karas 5.Cathy Garcia-Molina

**Top Actors on Netflix are:**

1Anupam Kher
2.Shah Rukh Khan 3.Naseeruddin Shah 4.Om Puri 5.Akshay Kumar


Most movies on Netflix have a duration range from 90 to 110 minutes
Most TV shows on Netflix have a span of only one season

#2. Analysis of Content produced in different countries

Drama is the most produced genre in a lot of Non-English speaking countries
Comedy is the most produced genre in English speaking countries like United States of America and United Kingdom and Canada
Drama and Comedy are the most produced genres in the top countries with exceptions of Japan and South Korea
Japan is the biggest producer of Anime. Anime is also the most produced in genre in Japan
Most South Korean content are from the Romance genre*
Documentaries are mainly produced in United Kingdom and United States of America

#3. Has Netflix been focusing increasingly on TV showss as compared to movies
From the first graph in the section, we can observe that seasons of TV shows signed vs the movies signed
This distinction gives contacts as TV shows require recurring investment for each seasons. So the TV numbers have been increased in accordance to the seasons. As they were considered as one entity earlier
We can observe that TV shows signed have been higher than movies in 2016. While the the movies signed have been higher, it is blatantly visible that the TV shows signed per year is catching up to the movies signed by the year

From the second graph in the section, we can observe an estimation of seasonal TV show and Movie release date per each year. We make a general assumption that each show releases a season yearly which is often the case for most TV shows.
There are certainly exemptions. Eg. Ozark was released in years 2017, 2018, 2020 and 2022.
We can observe that TV releases outnumbering movies from 2010. It is a known fact that each show contains multiple episodes, hence we can confidently say that significant amount of video content is being enlisted under TV shows from 2010
Years 2019, 2020 and 2021 see multi-episode TV seasons outnumbering movies signed per year

#4.1 Topic Modeling
Gensim Latent Drichlet was used to model textual data[description, genres, directors and cast] into topics
Nine topics were decided to be the most suitable from comparing Coherence Scores for different topic numbers.
The topics for each document will be used to classify the documents based on text as clustering can effectively handle so many values
The table also contains how much a document belongs to each topic, thus allowing content to be a mixture of different themes with their own respective topic weightages
By using this is as inputs, no data is lost in dimensionality reduction. The documents are classified based on their topics which is expected to be more sensible/insightful as an input
The Topics and corresponding top words are given down below:

Topic: 1

drama
comedy
famili
romance
find
young
woman
life
love
man

Topic: 2

drama
horror
thriller
romance
school
comedy
young
teen
student
life

Topic: 3

actionadventure
scififantasy
drama
sports
anime
team
world
kids
power
save

Topic: 4

comedy
drama
school
family
high
new
teen
world
student
scififantasy

Topic: 5

crime
drama
thriller
actionadventure
murder
investig
cop
polic
detect
drug

Topic: 6

documentary
documentari
music
seri
stori
explor
life
film
world
follow

Topic: 7

comedy
drama
romance
new
find
year
life
love
get
friend

Topic: 8

drama
comedy
romance
love
famili
man
young
woman
life
two

Topic: 9

comedy
kids
reality
friend
family
comedi
special
show
stand
comedian

#4.2 Clustering Results

# DBSCAN
DBSCAN clustered the data into 8 clusters with a silhouette score 0.4664
The model also had a Davies-Bouding Index of 1.6201 and Calinski-Harbaz Score of 2510.7

#K-MEANS
From the elbow graph and silhouette analysis, the best model has a silhouette score of 0.4686 produced with 8 Clusters using K Means Algorithm
The model also had a Davies-Bouding Index of 0.8895 and Calinski-Harbaz Score of 2901.84

#Hierarchical Clustering
Using dendograms and comparing various distance thresholds, a distance of 20 produced the highest silhouette score of 0.4687 with 8 clusters
The model also had a Davies-Bouding Index of 0.8886 and Calinski-Harbaz Score of 2900.28


## With the above results we can conclude that K-Means and Hierarchical clustering have the best results with 8 clusters